In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [109]:
q = """
with playoffs_21_22 as
    (select '2021-22' as season, left(right(event_name,4),2) as round, event_name, datediff(day, date(add_date),event_date) as days_out,
         sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_historical_all_1718_2223
    where season = '2021-22' and event_name like '%PO%' and ticket_type in ('Singles', 'Nightly Suites')
    group by event_name, event_date, add_date),
playoffs_22_23 as
    (select '2022-23' as season, left(right(event_name,4),2) as round, event_name, datediff(day, date(add_datetime),date(event_date)) as days_out,
       sum(block_purchase_price) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_ticket_expanded_all_playoffs_2223
    where event_name in ('23POR1G1','23POR1G2','23POR1G3','23POR2G1','23POR2G2','23POR3G1','23POR3G2','23POR4G1','23POR4G2')
        and ticket_type in ('Singles', 'Nightly Suites')
    group by event_name, event_date, date(add_datetime)),
playoffs_23_24 as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name, cast(0 as int) as days_out,
       sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2324_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '23-24 Pl'
    group by product_description, days_out)
select *
from playoffs_21_22
union
select *
from playoffs_22_23
union
select *
from playoffs_23_24
order by season, round, event_name, days_out desc
"""
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [110]:
df

,season,round,event_name,days_out,gross_revenue,paid_seats
0,2021-22,R1,22POR1G1,29,154278.000000,2165
1,2021-22,R1,22POR1G1,28,10349.000000,160
2,2021-22,R1,22POR1G1,27,6703.000000,105
3,2021-22,R1,22POR1G1,26,4689.000000,72
4,2021-22,R1,22POR1G1,25,40003.000000,433
...,...,...,...,...,...,...
211,2022-23,R4,23POR4G2,0,129066.899597,334
212,2023-24,R1,24POR1G1,0,243950.980000,1190
213,2023-24,R1,24POR1G2,0,170726.760000,875
214,2023-24,R1,24POR1G3,0,157658.080000,741


In [111]:
cumdf = df.groupby(by = ['round','event_name'], axis = 0)[['gross_revenue','paid_seats']].cumsum().rename(columns = {'gross_revenue':'cum_gross_rev', 'paid_seats':'cum_num_seats'})
dfdf = pd.concat([df,cumdf], axis = 1)
dfdf['final_seats'] = dfdf.apply(lambda row: dfdf[(dfdf['event_name'] == row['event_name'])&(dfdf['days_out'] == 0)]['cum_num_seats'].item(), axis = 1)
dfdf['per_seats_in'] = [x/y for x,y in zip(dfdf['cum_num_seats'],dfdf['final_seats'])]
dfdf['num_seats_left'] = dfdf.apply(lambda row: row['final_seats'] - row['cum_num_seats']+ row['paid_seats'], axis = 1)
dfdf = dfdf[['season','round', 'event_name', 'days_out','gross_revenue','paid_seats','num_seats_left','per_seats_in']]
# dfdf = dfdf.groupby(by = ['season','round','days_out']).mean()
dfdf

,season,round,event_name,days_out,gross_revenue,paid_seats,num_seats_left,per_seats_in
0,2021-22,R1,22POR1G1,29,154278.000000,2165,5348,0.404824
1,2021-22,R1,22POR1G1,28,10349.000000,160,3183,0.434742
2,2021-22,R1,22POR1G1,27,6703.000000,105,3023,0.454375
3,2021-22,R1,22POR1G1,26,4689.000000,72,2918,0.467838
4,2021-22,R1,22POR1G1,25,40003.000000,433,2846,0.548803
...,...,...,...,...,...,...,...,...
211,2022-23,R4,23POR4G2,0,129066.899597,334,334,1.000000
212,2023-24,R1,24POR1G1,0,243950.980000,1190,1190,1.000000
213,2023-24,R1,24POR1G2,0,170726.760000,875,875,1.000000
214,2023-24,R1,24POR1G3,0,157658.080000,741,741,1.000000


In [112]:
#df = df.merge(dfdf, how = 'left', on = ['season','round', 'days_out'])
FLA_Redshift(**rs_creds).write_to_warehouse(df = dfdf, table_name= "cth_historical_tier_velocity_playoffs")
df

,season,round,event_name,days_out,gross_revenue,paid_seats
0,2021-22,R1,22POR1G1,29,154278.000000,2165
1,2021-22,R1,22POR1G1,28,10349.000000,160
2,2021-22,R1,22POR1G1,27,6703.000000,105
3,2021-22,R1,22POR1G1,26,4689.000000,72
4,2021-22,R1,22POR1G1,25,40003.000000,433
...,...,...,...,...,...,...
211,2022-23,R4,23POR4G2,0,129066.899597,334
212,2023-24,R1,24POR1G1,0,243950.980000,1190
213,2023-24,R1,24POR1G2,0,170726.760000,875
214,2023-24,R1,24POR1G3,0,157658.080000,741
